<a href="https://colab.research.google.com/github/Uniholder/DeepLearningSchool/blob/main/2_semester/6_Transformers/%5Bhomework_part1%5DGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# !pip install -q transformers datasets tokenizers

     |████████████████████████████████| 3.1 MB 14.4 MB/s 
     |████████████████████████████████| 290 kB 67.8 MB/s 
     |████████████████████████████████| 3.3 MB 59.8 MB/s 
     |████████████████████████████████| 596 kB 55.3 MB/s 
     |████████████████████████████████| 895 kB 60.7 MB/s 
     |████████████████████████████████| 59 kB 9.2 MB/s 
     |████████████████████████████████| 132 kB 71.8 MB/s 
     |████████████████████████████████| 1.1 MB 40.0 MB/s 
     |████████████████████████████████| 243 kB 76.0 MB/s 
     |████████████████████████████████| 192 kB 75.0 MB/s 
     |████████████████████████████████| 160 kB 70.6 MB/s 
     |████████████████████████████████| 271 kB 72.8 MB/s 


# Домашнее задание. Transformers.

Привет! Это очередное домашнее задание, но теперь ты познакомишься с моделя трансформеров и с библиотекой `HuggingFace🤗`. В этом задании будет предложено решить с помощью модели `GPT2` простую задачу (анализ сентимента) и сделать небольшое исследование карт внимания. Приступим!

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker

import torch
import torch.nn as nn

from transformers import GPT2ForSequenceClassification, GPT2TokenizerFast, GPT2Config
from datasets import load_dataset

In [2]:
device = "cuda" if torch.cuda.is_available else "cpu"
device

'cuda'

Датасет, который мы будем использовать сегодня – тексты из английского твиттера. Они уже почищенны от никнеймов, знаков препинания и прочего. 

In [3]:
emotion_dataset = load_dataset("emotion")

Using custom data configuration default
Reusing dataset emotion (/root/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705)


  0%|          | 0/3 [00:00<?, ?it/s]

Посмотри, из чего состоит `emotion_dataset`:

In [6]:
emotion_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 2000
    })
})

In [7]:
emotion_dataset["train"]

Dataset({
    features: ['text', 'label'],
    num_rows: 16000
})

In [8]:
emotion_dataset["train"]["text"][0]

'i didnt feel humiliated'

In [9]:
emotion_dataset["train"]["label"][0]

0

In [10]:
len(emotion_dataset["train"])

16000

Для перевода текста в токены мы будем использовать предобученный BPE-токенайзер.

In [4]:
tokenizer = GPT2TokenizerFast.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token # У gpt2 нет pad токенов. Вместо них воспользуемся токенами конца текста.

Подготовь класс, который принимает датасет, токенайзер и имя используемой части (`train`, `validation`, `test`). Используй его для получения данных для обучения.

P.S. Посмотри, как работает токенайзер ([docs](https://huggingface.co/transformers/main_classes/tokenizer.html)) и подумай, как его надо добавить в датасет.

Немного примеров, как работает токенайзер. Это поможет с написанием датасета.

In [15]:
tokenizer.tokenize(emotion_dataset["train"]["text"][0])

['i', 'Ġdidnt', 'Ġfeel', 'Ġhumiliated']

In [31]:
emotion_dataset["train"]["text"][0]

'i didnt feel humiliated'

In [16]:
tokenizer.encode(emotion_dataset["train"]["text"][0])

[72, 42547, 1254, 42659]

In [17]:
tokenizer.encode_plus(emotion_dataset["train"]["text"][0])

{'input_ids': [72, 42547, 1254, 42659], 'attention_mask': [1, 1, 1, 1]}

In [18]:
tokenizer.encode_plus(emotion_dataset["train"]["text"][0], return_tensors="pt")

{'input_ids': tensor([[   72, 42547,  1254, 42659]]), 'attention_mask': tensor([[1, 1, 1, 1]])}

In [19]:
tokenizer.encode_plus(
    emotion_dataset["train"]["text"][0], 
    max_length=128, # максимальная длина текста
    padding="max_length", # надо ли добавлять паддинг в конце?
    return_tensors="pt", # возвращает pytorch тензоры
)

{'input_ids': tensor([[   72, 42547,  1254, 42659, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256, 50256,
         50256, 50256, 50256, 50256, 5

In [5]:
class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, part, dataset=emotion_dataset, tokenizer=tokenizer, max_length=128):
        self.part = part
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_length = max_length
        
        self.labels = np.unique(dataset[part]["label"])
        self.label2num = {l: num for num, l in enumerate(self.labels)}
        
    def __getitem__(self, idx):
        """
        Returns dict with tokens, attention_mask and label
        """
        text = self.dataset[self.part]['text'][idx]
        label = self.dataset[self.part]['label'][idx]
        
        tokenizer_output = self.tokenizer.encode_plus(
            text=text,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        target = self.label2num[label]
        return {
            "input_ids":  tokenizer_output['input_ids'].squeeze(),
            "mask": tokenizer_output['attention_mask'].squeeze(),
            "target": target
        }
        
    def __len__(self):
        """
        Returns length of dataset
        """
        return len(self.dataset[self.part])

Создай `train`, `validation` и `test` части датасета. Загрузи их в `DataLoaders`.

In [6]:
train_dataset = TweetDataset('train', emotion_dataset, tokenizer, 128)
valid_dataset = TweetDataset('validation', emotion_dataset, tokenizer, 128)
test_dataset = TweetDataset('test', emotion_dataset, tokenizer, 128)

In [7]:
batch_size = 128

train_loader = torch.utils.data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True
)
valid_loader = torch.utils.data.DataLoader(
    dataset=valid_dataset,
    batch_size=batch_size,
    shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    shuffle=True
)

## Начнем с нуля.

Попробуем обучить модель трансформер с нуля решать данную задачу.

In [9]:
config = GPT2Config.from_pretrained(
    "distilgpt2",  # distilgpt2 – уменьшенная версия модели gpt2
    output_attentions=True,
    pad_token_id=tokenizer.eos_token_id,
    num_labels=8
)
model_0 = GPT2ForSequenceClassification(config=config).to(device)  # GPT2 для классификации текста

Подготовь оптимайзер и критерий:

In [31]:
lr = 1e-4

optimizer = torch.optim.Adam(model_0.parameters(), lr)
criterion = nn.CrossEntropyLoss()
# scheduler = torch.optim.lr_scheduler.StepLR()

Посмотри, что возвращает модель ([docs](https://huggingface.co/transformers/model_doc/gpt2.html#gpt2forsequenceclassification)), если в неё подать данные:

In [12]:
tokenized = tokenizer.encode_plus(
    emotion_dataset["train"]["text"][0], 
    max_length=128,  # максимальная длина текста
    padding="max_length",  # надо ли добавлять паддинг в конце?
    return_tensors="pt",  # возвращает pytorch тензоры
)
tokens = tokenized['input_ids'].to(device)
mask = tokenized['attention_mask'].to(device)

output = model_0(tokens, attention_mask=mask)

In [14]:
output.logits

tensor([[ 0.6717, -0.4686,  0.1700,  0.1531, -0.0961, -0.6903, -1.4931,  0.8655]],
       device='cuda:0', grad_fn=<IndexBackward0>)

Обучи модель с помощью `train_dataset`, проверяй во время обучения качество с помощью `valid_dataset` и получи финальное качество с помощью `test_dataset`.

In [32]:
from tqdm.notebook import tqdm


num_epochs = 1#0

# Train loop
for e in range(num_epochs):
    model_0.train()
    train_loss = 0
    for batch in tqdm(train_loader):
        input_ids = batch['input_ids'].to(device)
        mask = batch['mask'].to(device)
        target = batch['target'].to(device)

        optimizer.zero_grad()

        outputs = model_0(input_ids, attention_mask=mask)
        logits = outputs.logits
        # break
        loss = criterion(logits, target)
        loss.backward()

        optimizer.step()

        train_loss += loss.item()

    # break
    valid_loss = 0
    valid_acc = 0
    model_0.eval()
    with torch.no_grad():
        for batch in tqdm(valid_loader):
            input_ids = batch['input_ids'].to(device)
            mask = batch['mask'].to(device)
            target = batch['target'].to(device)

            outputs = model_0(input_ids, attention_mask=mask)
            logits = outputs.logits

            loss = criterion(logits, target)
            valid_loss += loss
            valid_acc += torch.sum(torch.argmax(logits, dim=1) == target) / len(batch)

    print(f"Train Loss: {train_loss / len(train_loader)}, "
          f"Valid Loss: {valid_loss / len(valid_loader)}, "
          f"Valid Acc: {valid_acc / len(valid_loader)}")
    
# Testing
# test_acc = 0
# model_0.eval()
# with torch.no_grad():
#     for batch in test_loader:
#         batch = batch.to(device)
#         tokens, mask, target = batch

#         outputs = model_0(tokens, attention_mask=mask)
#         logits = outputs.logits

#         valid_loss += loss
# print(f"Test Acc: {test_acc / len(test_loader)}")

  0%|          | 0/125 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

RuntimeError: ignored

In [45]:
valid_loss = 0
valid_acc = 0
model_0.eval()
with torch.no_grad():
    for batch in tqdm(valid_loader):
        input_ids = batch['input_ids'].to(device)
        mask = batch['mask'].to(device)
        target = batch['target'].to(device)

        outputs = model_0(input_ids, attention_mask=mask)
        logits = outputs.logits

        loss = criterion(logits, target)
        valid_loss += loss
        valid_acc += torch.sum(torch.argmax(logits, dim=1) == target) / len(batch['input_ids'])

  0%|          | 0/16 [00:00<?, ?it/s]

In [46]:
print(f"Train Loss: {train_loss / len(train_loader)}",
          f"Valid Loss: {valid_loss / len(valid_loader)}",
          f"Valid Acc: {valid_acc / len(valid_loader)}")

Train Loss: 1.5683060278892518 Valid Loss: 1.0575942993164062 Valid Acc: 0.601269543170929


После того, как получил лучшую модель для решения этой задачи, посмотри на карты внимания. Нашел ли что-нибудь интересное в них?

In [ ]:
def get_attention_matrixes(model, tokenizer, text, device=device):
    inp = list(filter(lambda x: x != tokenizer.sep_token_id, tokenizer.encode(text)))
    inp = torch.tensor(inp, dtype=torch.long, device=device).unsqueeze(0)
    attn_tensors = model(inp)[-1]
    seq = [tokenizer.decode(x) for x in inp[0].tolist()]
    attn = []
    for i in range(len(attn_tensors)):
        attn_layer = []
        for j in range(attn_tensors[i].size(1)):
            attn_layer.append(attn_tensors[i][0, j].cpu().detach().numpy())
        attn.append(np.array(attn_layer))
    
    return np.array(attn)

In [ ]:
def show_attention(seq, attentions):
    # Set up figure with colorbar
    fig = plt.figure(figsize=(20,20))
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions)
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels(['']+seq, rotation=90, fontsize=16)
    ax.set_yticklabels(['']+seq, fontsize=16)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()

In [ ]:
text = ... # Выбери текст из датасета
tokens = tokenizer.tokenize(text)

In [ ]:
attns = get_ettention_matrixes(model_0, tokenizer, text)
show_attention(seq, attn[-1][0])

## Fine-tuning

Теперь другой подход: загрузим модель, которая обучалась решать задачу Language Modeling. Посмотрим, получим ли мы прирост в качестве.

In [ ]:
model_1 = GPT2ForSequenceClassification.from_pretrained(
    "distilgpt2", 
    output_attentions=True,
    pad_token_id=tokenizer.eos_token_id,
    num_labels=8
).to(device)

In [ ]:
lr = 1e-5 # Предполагаемый learning rate. Он может быть больше или меньше :)

optimizer = ...
criterion = ...
# scheduler = ... # Можно добавить шедулер для обучения моделей. Это на твое усмотрение

Вывод модели ничем не отличается от предыдущего случая, поэтому сразу приступаем к обучению:

In [ ]:
from tqdm.notebook import tqdm


num_epochs = 10

# Train loop
for e in range(num_epochs):
    model_1.train()
    train_loss = 0
    for batch in tqdm(train_loader):
        ...
        
    valid_loss = 0
    valid_acc = 0
    model_1.eval()
    with torch.no_grad():
        for batch in valid_loader:
            ...
    print(f"Train Loss: {train_loss / len(train_loader)},"
          f"Valid Loss: {valid_loss / len(valid_loader)},"
          f"Valid Acc: {valid_acc / len(valid_loader)}")
        
# Testing
test_acc = 0
model_1.eval()
with torch.no_grad():
    for batch in test_loader:
        ...
print(f"Test Acc: {test_acc / len(test_loader)}")

Есть ли прирост качества или скорости обучения?

Посмотри на карты внимания. Есть ли отличие от предыдущего случая?

In [ ]:
text = ... # Выбери текст из датасета
tokens = tokenizer.tokenize(text)

In [ ]:
attns = get_ettention_matrixes(model, tokenizer, text)
show_attention(seq, attn[-1][0])

## Отчет

Покажи здесь, что ты выполнил по этой работе. Ответь на несколько вопросов:
- Какой подход оказался лучше? 
- На какие слова модель большего всего обращала внимание?
- На каких слоях/головах модель обращала внимание?

< твой отчет/ответы >